# Métricas do câmbio do dólar (2025)

## About

Este projeto tem como objetivo gerar estatísticas básicas para analisar a flutuação de valor do dólar em relação ao real durante o ano de 2025. Para isso, foram utilizadas técnicas de webscraping para obter a série histórica do dólar para o ano. Uma vez obtidos, os valores foram adicionados a uma planilha excel salva em nuvem, que por sua vez está vinculada a um dashboard Google Looker, onde foram geradas as estatísticas. 

O projeto está estruturado de forma a sempre os últimos valores divulgados, portanto, para atualizar o dashboard, basta rodar o código abaixo:


In [1]:
import requests
import pandas as pd
from datetime import datetime


In [ ]:

# scrape_to_excel.py
# Fetch the table from https://idealsoftwares.com.br/indices/dolar2025.html and save to a local Excel file.


URL = "https://idealsoftwares.com.br/indices/dolar2025.html"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; GitHub Copilot)"}

def fetch_tables(url):
    resp = requests.get(url, headers=HEADERS, timeout=15)
    resp.raise_for_status()
    # let pandas detect encoding from the HTML text
    tables = pd.read_html(resp.text)
    return tables

def choose_table(tables):
    if not tables:
        raise ValueError("No tables found on the page.")
    # choose the most likely table (first). Adjust index if needed.
    return tables[0]

def clean_table(df: pd.DataFrame) -> pd.DataFrame:
    # basic clean: strip string columns and drop fully empty columns/rows
    df = df.dropna(axis=1, how="all").dropna(axis=0, how="all")
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip()
    return df

def save_to_excel(df: pd.DataFrame, prefix="dolar2025"):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{prefix}_{ts}.xlsx"
    df.to_excel(filename, index=False)
    return filename

if __name__ == "__main__":
    tables = fetch_tables(URL)
    df = choose_table(tables)
    df = clean_table(df)
    out_file = save_to_excel(df)
    print(f"Saved table to: {out_file}")
    print(df.head())